## Desafío - Regresión desde el aprendizaje de maquinas
### Camila Vera Villa - miercoles 24 de julio

#### Contexto
En esta sesión trabajaremos una base de datos sobre los precios de las viviendas en Boston,
utilizada en el paper Harrison Jr, D., & Rubinfeld, D. L. (1978). Hedonic housing prices and the
demand for clean air. Journal of environmental economics and management, 5(1), 81-102.
Nuestro objetivo es desarrollar un modelo predictivo para el valor mediano de las casas mediante el
entrenamiento de un modelo de regresión lineal.


* crim : Tasa de criminalidad por sector de Boston
* zn: proporción de terreno residencial asignado para terrenos baldíos.
* indus: proporción de negocios no asociados al comercio por sector.
* chas: Dummy. 1 si el sector colinda con el río Charles, 0 de lo contrario.
* nox: Concentración de dióxido de carbono
* rm: cantidad promedio de habitaciones por casa.
* age: proporción de casas construídas antes de 1940
* dis: distancia promedio a cinco centros de empleos.
* rad: índice de accesibilidad a autopistas.
* tax: nivel de impuestos asociados a viviendas.
* ptratio: razón alumno:profesor por sector de Boston.
* black: proporción de afroamericanos por sector de Boston.
* lstat: porcentaje de población de estratos bajos.
* medv: valor mediano de las casas

### Desafío 1: Prepare el ambiente de trabajo

* Importe las librerías básicas para el análisis de datos.
* Importe el módulo linear_model , y las funciones mean_squared_error, r2_score y train_test_split.
* Importe la base de datos boston.csv y elimine la columna Unnamed: 0.
* Obtenga las medidas descriptivas de la base de datos con .describe().

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Eliminacion de la primera columna
df = pd.read_csv('boston.csv').drop('Unnamed: 0', axis=1) 
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
#Medidas descriptivas del df
df.describe() 

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


### Desafío 2: División de la muestra

* Genere conjuntos de entrenamiento y validación con train_test_split.
* Genere una segmentación del 33% para las muestras de validación.
* Incluya una semilla pseudoaleatoria

In [4]:
# seed = 17955

In [5]:
#generacion de matriz de atributos y vector objetivo -> medv: valor mediano de las casas

mat_atrib = df.drop('medv', axis = 1) #matriz que excluya vector objetivo
y_vector = df['medv']


In [6]:
  x_train, x_test, y_train, y_test = train_test_split(mat_atrib, y_vector, 
                                                      test_size = 0.33,random_state = 17955)

### Desafío 3: Generación de modelos
Ahora implementaremos dos versiones del modelo lineal:
* Con intercepto y atributos normalizados.
* Sin intercepto y atributos no normalizados.
* Cada versión debe generarse en un nuevo objeto inicializado.
* Posteriormente se deben entrenar los modelos especificando la matriz y vector de entrenamiento.
* Con los modelos entrenados, genere una predicción con su matriz de validación.

In [7]:
#generacion de 2 versiones del modelo lineal en un nuevo objeto
modelo1 = linear_model.LinearRegression(fit_intercept= True, normalize = True)
modelo2 = linear_model.LinearRegression(fit_intercept= False, normalize = False)

In [8]:
#entrenar modelos
modelo1.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [9]:
modelo2.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [10]:
x_test.shape

(167, 13)

In [11]:
#prediccion del modelo, se guarda un y nuevo para compararlo con el y real
predict1 = modelo1.predict(x_test)  
len(predict1)

167

In [12]:
#prediccion del modelo
predict2 = modelo2.predict(x_test)

### Desafío 4: Obtención de métricas
* Ahora generemos una función llamada report_scores que ingrese como argumentos el vector de datos predichos y el vector de datos por validar.

* La función debe imprimir las métricas del Error Cuadrático Promedio y R2. 

* Reporte las métricas para ambos modelos. En base a ello, seleccione el mejor modelo.

In [13]:
def report_scores(vector_datos_predichos, vector_datos_porvalidar):
    '''Def: Función que genere reportes de un modelo sobre un conjunto de datos predichos y de datos por validar
    Input: 2 vectores que contienen los datos predichos y los datos por validar
    Output: la función imprime las metricas del Error Cuadrático Promedio(MSE) y R2.'''

    mse = mean_squared_error(vector_datos_predichos,vector_datos_porvalidar).round(3)
    r2 = r2_score(vector_datos_predichos,vector_datos_porvalidar).round(3)
    
    print('El MSE es:  {}'.format(mse))
    print('EL R2 es: {}'.format(r2))


In [14]:
# REPORTE DE METRICAS MODELO 1
report_scores(y_test, predict1)


El MSE es:  20.718
EL R2 es: 0.766


In [15]:
# REPORTE DE METRICAS MODELO 2
report_scores(y_test,predict2)

El MSE es:  21.241
EL R2 es: 0.76


El mejor modelo corresponde al modelo 1, ya que el Promedio de Error Cuadratico (20.7183) es menor que el del modelo 2 (21.2408). A pesar de que los R2 favorecen mas al modelo 2, sin emavrgo de manera leve.

### Desafío 5: Refactorización del modelo
* Genere una función llamada fetch_features que ingrese como argumentos la base de datos (df) y el nombre del vector objetivo. El nombre del vector debe ser medv por defecto
* La función debe retornar una lista con las correlaciones entre cada atributo y el vector objetivo y su nombre.
* Reporte brevemente cuales los 6 atributos con una mayor correlación con medv

In [16]:
def fetch_features(df, vector_objetivo = 'medv'):
    '''Def: Función que permite obtener las correlaciones de Pearson del vector ingresado en comparación al resto del dataframe
    Input:dataframe de origen y vector_objetivo='medv, para este caso por defecto es el valor mediano de las propiedades
    Outpout: retorna una lista con con las correlaciones entre cada atributo y el vector objetivo y su nombre.'''
     
    #generacion de arrays vacios para guardar los valores
        
    columns = df.columns   #columnas de las base de datos
    nombre_atrib = []      #nombre de la variable
    pearson_r = []         #correlacion de Pearson
    abs_pearson_r = []     #valor absoluto de la correlacion
    
    #loop para cada columna en el array de columnas
    
    for col in columns:
        if col != vector_objetivo:    #si la columna no es la dependiente
            nombre_atrib.append(col) # adjuntar nombre de la variable en nombre_atrib
            pearson_r.append(df[col].corr(df[vector_objetivo])) #adjuntar la correlacion de pearson
            abs_pearson_r.append(abs(df[col].corr(df[vector_objetivo])))
            
    #transformar array en df
    
    caracteristicas = pd.DataFrame({
        'atributo': nombre_atrib,
        'correlacion': pearson_r,
        'valor_abs_pearson': abs_pearson_r})
    
    #generacion del index con los nombres de las variables
    caracteristicas = caracteristicas.set_index('atributo')
        
    #orden de los valores de forma descendiente
    return caracteristicas.sort_values(by=['valor_abs_pearson'], ascending = False)        
          

In [17]:
fetch_features(df)

,correlacion,valor_abs_pearson
atributo,,
lstat,-0.737663,0.737663
rm,0.695360,0.695360
ptratio,-0.507787,0.507787
indus,-0.483725,0.483725
tax,-0.468536,0.468536
nox,-0.427321,0.427321
crim,-0.388305,0.388305
rad,-0.381626,0.381626
age,-0.376955,0.376955


  * Los 6 atributos con una mayor correlación con medv, son:
1. nox
2. tax
3. indus
4. ptratio
5. rm
6. istat

### Desafío 6: Refactorización del modelo predictivo
* Genere otros conjuntos de entrenamiento y validación en base a una matriz con los 6 atributos identificados y el vector objetivo.
* Entrene un modelo en base al mejor desempeño.
* Reporte las métricas para el nuevo modelo.

In [18]:
#generacion de matriz de atributos y vector objetivo -> medv: valor mediano de las casas

x_matriz = df.loc[:,['lstat', 'rm', 'ptratio', 'indus', 'tax', 'nox']] #matriz con 6 atributos
y_vector_obj = df['medv'] # vector objetivo


In [19]:
#generacion de conjuntos de entrenamiento
    
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(x_matriz, y_vector_obj, 
                                                      test_size = 0.33,random_state = 17955)

In [20]:
#generacion del modelo lineal con los 6 atributos anteriores y vector objetivo: medv

modelo3 = linear_model.LinearRegression(fit_intercept= True, normalize = True)

#entrenar modelos

modelo3.fit(x_train_2,y_train_2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [21]:
#metricas modelo 3 y listado de coeficientes e intercepto

print('LISTADO DE COEFICIENTES E INTERCEPTO DEL MODELO 3')
print('\n')
print('La lista de pendientes es: ', x_matriz.columns)
print('\n')
print('Valores de los coeficientes de las pendientes: ', modelo3.coef_)
print('\n')

print('El intercepto del modelo es: ', modelo3.intercept_)


LISTADO DE COEFICIENTES E INTERCEPTO DEL MODELO 3


La lista de pendientes es:  Index(['lstat', 'rm', 'ptratio', 'indus', 'tax', 'nox'], dtype='object')


Valores de los coeficientes de las pendientes:  [-0.57921803  3.68977199 -0.89235266  0.10861109 -0.0045254  -2.30151821]


El intercepto del modelo es:  24.96037487368618


In [22]:
#prediccion del modelo
predict3 = modelo3.predict(x_test_2)
print('La cantidad de predicciones realizadas para el modelo 3 son: ', len(predict3))
print('\n')
print('Las predicciones son: ', predict3.round(2))

La cantidad de predicciones realizadas para el modelo 3 son:  167


Las predicciones son:  [16.28 13.79 18.34 32.7  21.32 22.17 21.85 14.41  4.2  23.69 22.43 32.53
 22.87 30.3  23.2  19.9  37.58 29.57 36.93 25.41 18.98 36.88 27.64 26.52
 24.31 27.38 22.76 16.71 24.63 31.03 17.15 25.48 15.57 14.34 21.28  5.9
 18.04 19.31 28.21 27.57 12.58 19.71 27.74 19.53 14.27 16.06 24.56 27.1
 21.74 20.12 34.47 16.54 18.43 26.25 28.92 35.81 26.26 17.3  32.13 26.03
 28.61 22.01 19.93 16.78 23.57 17.57 33.65 30.12 24.46 17.05 11.9  20.02
 18.75 14.54 23.22 25.84 39.73 33.08 16.66 20.98 20.21 25.85  9.62 22.39
 33.79 30.95 33.57 18.   25.64 17.01 23.7  23.25 27.53 34.87 26.08 35.41
 32.39 27.22 21.54 15.4  21.38 22.94 24.11 26.93  2.31 34.65 27.97 18.25
 36.49 21.61 23.67 11.95 31.85 22.73 10.38 34.49 24.88 17.87 22.35 17.44
 20.1  21.83 27.58 21.89 22.06 22.32 22.28 15.92 29.73 11.69 28.04 29.37
 15.41 16.51 27.6  19.45 31.72 17.08 22.16 11.24 31.96 27.73 27.24 29.76
 38.14 19.08 12.87 14.36 38.67 15.1

In [23]:
report_scores(y_test_2, predict3)

El MSE es:  24.395
EL R2 es: 0.724


### Desafío 7: Predicción de casos
A continuación se generaron dos arrays que representan el peor escenario posible:

( worst_neighbor ) y el mejor escenario posible ( best_neighbor ).

* Ingrese los arrays en el modelo entrenado y reporte cuál sería el valor esperado dada las condiciones

    **worst_neighbor = np.array([37.9, 12.6, 3.5, 27.7, 187, 0.87]).reshape(1, -1)**

    **best_neighbor = np.array([1.73, 22, 8.7, 0.46, 711, 0.38]).reshape(1, -1)**

In [24]:
#peor vecindario
worst_neighbor = modelo3.predict(np.array([37.9, 12.6, 3.5, 27.7, 187, 0.87]).reshape(1, -1))
print('El valor esperado de las propiedades del peor vecindario es: ',worst_neighbor[0].round(3))

El valor esperado de las propiedades del peor vecindario es:  46.536


In [25]:
#mejor vecindario
best_neighbor = modelo3.predict(np.array([1.73, 22, 8.7, 0.46, 711, 0.38]).reshape(1, -1))
print('El valor esperado de las propiedades del mejor vecindaro: ', best_neighbor[0].round(3))

El valor esperado de las propiedades del mejor vecindaro:  93.328


  * Los valores representan a los 6 atributos con una mayor correlación con medv:
1. nox
2. tax
3. indus
4. ptratio
5. rm
6. istat